In [1]:
import os
import requests
import pandas as pd

ENV = os.environ.get("ENV", "dev")
if ENV == "test":
    DB_URL = "sqlite"
else:
    DB_URL = "postgres"

print(DB_URL)

# URL base de la API
BASE_URL = "http://localhost:8000" 

response = requests.get(f"{BASE_URL}/")
print(response.status_code, response.json())

postgres
200 {'message': 'API is running'}


In [2]:
df = pd.read_csv("data/hired_employees.csv", names=["id","name","datetime","department_id","job_id"], header=None, parse_dates=["datetime"], na_values=["NaN","N/A","null"])
# df = pd.read_csv("hired_employees.csv", names=["id","name","datetime","department_id","job_id"], header=None)

print("¿Hay NaN en 'id'?             ", df["id"].isna().any())
print("¿Hay NaN en 'department_id'?  ", df["department_id"].isna().any())
print("¿Hay NaN en 'job_id'?         ", df["job_id"].isna().any())
print("¿Hay NaN en 'datetime'?       ", df["datetime"].isna().any())
print("¿Hay NaN en 'name'?           ", df["name"].isna().any())
print("\n")

df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce", utc=True)
df["datetime"] = df["datetime"].dt.tz_convert(None)

nan_por_columna = df.isna().sum()
print(f"Cantidad de NaNs por columna: \n{nan_por_columna}\n")

total_nan = df.isna().sum().sum()
print(f"Total de valores NaN en el DataFrame: {total_nan}\n")


# # Approach 1: Rellena NaN en 'name'
# df["name"] = df["name"].fillna("Unknown")
# df["job_id"] = df["job_id"].fillna(0).astype(int)
# df["department_id"] = df["department_id"].fillna(0).astype(int)
# # Approach 2: Rellena NaN en 'name'
# df["name"] = df["name"].where(pd.notnull(df["name"]), "Unknown")
# df["job_id"] = df["job_id"].where(df["job_id"].notna(), 0).astype(int)
# df["department_id"] = df["department_id"].where(df["department_id"].notna(), 0).astype(int)
# # Approach 3
# df["job_id"] = df["job_id"].where(pd.notnull(df["job_id"]), 0)
# df["department_id"] = df["department_id"].where(pd.notnull(df["department_id"]), 0)

filas_antes = len(df)
df = df.dropna()
filas_despues = len(df)
filas_eliminadas = filas_antes - filas_despues
print(f"Filas antes: {filas_antes}, Filas después: {filas_despues}, Filas eliminadas: {filas_eliminadas}\n")

# df
print("¿Hay NaN en 'id'?             ", df["id"].isna().any())
print("¿Hay NaN en 'department_id'?  ", df["department_id"].isna().any())
print("¿Hay NaN en 'job_id'?         ", df["job_id"].isna().any())
print("¿Hay NaN en 'datetime'?       ", df["datetime"].isna().any())
print("¿Hay NaN en 'name'?           ", df["name"].isna().any())

# # Filtrar registros con valores no numéricos
# df_non_numeric_col = df[pd.to_numeric(df['department_id'], errors='coerce').isna()]

# df_dict = df.to_dict(orient="records")
# print(df_dict)

¿Hay NaN en 'id'?              False
¿Hay NaN en 'department_id'?   True
¿Hay NaN en 'job_id'?          True
¿Hay NaN en 'datetime'?        True
¿Hay NaN en 'name'?            True


Cantidad de NaNs por columna: 
id                0
name             19
datetime         14
department_id    21
job_id           16
dtype: int64

Total de valores NaN en el DataFrame: 70

Filas antes: 1999, Filas después: 1929, Filas eliminadas: 70

¿Hay NaN en 'id'?              False
¿Hay NaN en 'department_id'?   False
¿Hay NaN en 'job_id'?          False
¿Hay NaN en 'datetime'?        False
¿Hay NaN en 'name'?            False


In [3]:
file_path = "data/jobs.csv"

with open(file_path, "rb") as file:
    files = {"file": file}
    response = requests.post(f"{BASE_URL}/upload-jobs", files=files)

print("\n✅ Upload CSV Response:", response.status_code, response.text)


✅ Upload CSV Response: 200 {"message":"Jobs uploaded successfully"}


In [4]:
file_path = "data/departments.csv"

with open(file_path, "rb") as file:
    files = {"file": file}
    response = requests.post(f"{BASE_URL}/upload-departments", files=files)

print("\n✅ Upload CSV Response:", response.status_code, response.text)


✅ Upload CSV Response: 200 {"message":"Departments uploaded successfully"}


In [5]:
file_path = "data/hired_employees.csv"

with open(file_path, "rb") as file:
    files = {"file": file}
    response = requests.post(f"{BASE_URL}/upload-employees-dropna", files=files)

print("\n✅ Upload CSV Response:", response.status_code, response.text)


✅ Upload CSV Response: 200 {"message":"Employees uploaded successfully"}


In [6]:
response = requests.get(f"{BASE_URL}/departments")
if response.status_code == 200:
    departments = response.json()
    # print("\n✅ Departments:", json.dumps(departments, indent=4))
    departments_df = pd.DataFrame(departments)
    print(departments_df.head(5))
else:
    print(f"Error: {response.status_code}")

   id                department
0   1        Product Management
1   2                     Sales
2   3  Research and Development
3   4      Business Development
4   5               Engineering


In [7]:
response = requests.get(f"{BASE_URL}/jobs")
if response.status_code == 200:
    jobs = response.json()
    # print("\n✅ jobs:", json.dumps(jobs, indent=4))
    jobs_df = pd.DataFrame(jobs)
    print(jobs_df.head(5))
else:
    print(f"Error: {response.status_code}")

   id                        job
0   1        Marketing Assistant
1   2                   VP Sales
2   3         Biostatistician IV
3   4  Account Representative II
4   5               VP Marketing


In [8]:
response = requests.get(f"{BASE_URL}/employees")
if response.status_code == 200:
    employees = response.json()
    # print("\n✅ employees:", json.dumps(employees, indent=4))
    employees_df = pd.DataFrame(employees)
    print(employees_df.head(5))
else:
    print(f"Error: {response.status_code}")

   department_id  id            name             datetime  job_id
0              2   1     Harold Vogt  2021-11-07T02:48:42      96
1              5   3     Lyman Hadye  2021-09-01T23:27:38      52
2             12   4   Lotti Crowthe  2021-10-01T13:04:21      71
3              6   5  Gretna Lording  2021-10-10T22:22:17      80
4              6   6  Marlow Antecki  2021-04-23T23:45:42      95


In [9]:
# Endpoint de empleados por ID
EMPLOYEE_ID = 1
response = requests.get(f"{BASE_URL}/employees/{EMPLOYEE_ID}")
employees_by_id_df = pd.json_normalize(response.json())
# print("\n✅ Employee by ID:", json.dumps(response.json(), indent=4))
print("\n✅ Employee by ID \n", employees_by_id_df)


✅ Employee by ID 
    id         name             datetime  department_id  job_id
0   1  Harold Vogt  2021-11-07T02:48:42              2      96


In [10]:
# Endpoint de empleados por departamento
DEPARTMENT_ID = 1
response = requests.get(f"{BASE_URL}/employees/department/{DEPARTMENT_ID}")
employees_by_department_df = pd.DataFrame(response.json())
# print("\n✅ Employees by Department:", json.dumps(response.json(), indent=4))
print("\n✅ Employees by Department \n", employees_by_department_df)


✅ Employees by Department 
       id                 name             datetime  department_id  job_id
0     81         Caril Bewsey  2022-01-22T05:56:43              1      73
1    175          Billy Odams  2021-02-13T21:13:22              1      59
2    179      Wolfy Leicester  2021-02-11T23:08:12              1     137
3    223      Malvina Camelli  2021-03-03T05:07:46              1      83
4    227        Jeramey Pyson  2021-04-03T20:14:34              1      43
5    246       Neddy O'Garmen  2021-06-29T18:44:50              1      85
6    263   Eldredge Behnecken  2021-06-25T00:23:17              1     101
7    284     Sile Alexsandrev  2021-10-25T03:10:45              1      66
8    323           Erv Hubane  2021-05-03T12:04:54              1      14
9    337       Rosemarie Sofe  2021-11-07T08:25:36              1     124
10   345       Corenda Dubbin  2021-03-06T14:32:27              1     136
11   351        Weston Rouchy  2021-10-29T06:10:25              1      26
12   430 

In [11]:
# Endpoint de empleados por trabajo
JOB_ID = 3
response = requests.get(f"{BASE_URL}/employees/job/{JOB_ID}")
employees_by_job_df = pd.DataFrame(response.json())
# print("\n✅ Employees by Department:", json.dumps(response.json(), indent=4))
print("\n✅ Employees by Department \n", employees_by_job_df)


✅ Employees by Department 
      id                name             datetime  department_id  job_id
0    73   Matthus Szymanzyk  2021-04-26T04:21:25              7       3
1  1684  D'arcy O'Kinneally  2021-07-01T14:02:42              7       3


In [12]:
response = requests.get(f"{BASE_URL}/{DB_URL}/employees-per-quarter")
employees_per_quarter_df = pd.DataFrame(response.json())
# print("\n✅ Employees by Department:", json.dumps(response.json(), indent=4))
print("\n✅ Employees per Quarter:\n", employees_per_quarter_df)


✅ Employees per Quarter:
      department                            job  q1  q2  q3  q4
0    Accounting      Account Representative IV   1   0   0   0
1    Accounting                        Actuary   0   1   0   0
2    Accounting             Analyst Programmer   0   0   1   0
3    Accounting  Budget/Accounting Analyst III   0   1   0   0
4    Accounting                Cost Accountant   0   1   0   0
..          ...                            ...  ..  ..  ..  ..
928    Training                        Teacher   0   2   0   0
929    Training               Technical Writer   0   0   1   0
930    Training          VP Product Management   1   0   0   0
931    Training             VP Quality Control   0   1   0   0
932    Training              Web Developer III   0   1   0   0

[933 rows x 6 columns]


In [13]:
response = requests.get(f"{BASE_URL}/{DB_URL}/departments-hired-above-mean")
departments_hired_above_mean_df = pd.DataFrame(response.json())
# print("\n✅ Employees by Department:", json.dumps(response.json(), indent=4))
print("\n✅ Employees per Quarter:\n", departments_hired_above_mean_df)


✅ Employees per Quarter:
    id                department  hired
0   8                   Support    216
1   5               Engineering    205
2   6           Human Resources    201
3   7                  Services    200
4   4      Business Development    185
5   3  Research and Development    148
6   9                 Marketing    142


In [19]:
import urllib.parse  # Para codificar el query

custom_sql = """
SELECT 
    he.name,
    he.datetime,
    j.job,
    d.department
FROM hired_employees he
JOIN departments d ON he.department_id = d.id
JOIN jobs j ON he.job_id = j.id
limit 5
"""

encoded_query = urllib.parse.quote(custom_sql)

response = requests.get(f"{BASE_URL}/custom-query/?query={encoded_query}")

# Convertir la respuesta a DataFrame si es exitosa
if response.status_code == 200:
    results_df = pd.DataFrame(response.json())
    print("\n✅ Custom Query Result:\n", results_df)
else:
    print(f"\n❌ Error {response.status_code}: {response.text}")


✅ Custom Query Result:
              name             datetime                           job  \
0     Harold Vogt  2021-11-07T02:48:42              Health Coach III   
1     Lyman Hadye  2021-09-01T23:27:38  Structural Analysis Engineer   
2   Lotti Crowthe  2021-10-01T13:04:21               Statistician II   
3  Gretna Lording  2021-10-10T22:22:17              Quality Engineer   
4  Marlow Antecki  2021-04-23T23:45:42               Web Designer II   

        department  
0            Sales  
1      Engineering  
2       Accounting  
3  Human Resources  
4  Human Resources  
